## [SQLAlchemy チュートリアル概要｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/bee9e4)

In [1]:
import sqlalchemy
sqlalchemy.__version__

'2.0.39'

## [接続の確立 - エンジンの作成｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/754e70)

In [2]:
from sqlalchemy import create_engine

# SQLiteのインメモリデータベースに接続するEngineを作成
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

## [トランザクションとDBAPIの操作｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/f91303)

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2025-03-14 14:51:23,596 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,598 INFO sqlalchemy.engine.Engine select 'hello world'
2025-03-14 14:51:23,599 INFO sqlalchemy.engine.Engine [generated in 0.00299s] ()
[('hello world',)]
2025-03-14 14:51:23,600 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()

2025-03-14 14:51:23,611 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,613 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-03-14 14:51:23,613 INFO sqlalchemy.engine.Engine [generated in 0.00193s] ()
2025-03-14 14:51:23,616 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-03-14 14:51:23,617 INFO sqlalchemy.engine.Engine [generated in 0.00113s] [(1, 1), (2, 4)]
2025-03-14 14:51:23,618 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2025-03-14 14:51:23,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,630 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-03-14 14:51:23,631 INFO sqlalchemy.engine.Engine [cached since 0.0152s ago] [(6, 8), (9, 10)]
2025-03-14 14:51:23,632 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-03-14 14:51:23,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,647 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-03-14 14:51:23,648 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()
x: 1, y: 1
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2025-03-14 14:51:23,650 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 5})

    for row in result:
        print(f"x: {row.x}, y: {row.y}")

print(type(result))

2025-03-14 14:51:23,662 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,664 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2025-03-14 14:51:23,665 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (5,)
x: 6, y: 8
x: 9, y: 10
2025-03-14 14:51:23,666 INFO sqlalchemy.engine.Engine COMMIT
<class 'sqlalchemy.engine.cursor.CursorResult'>


In [8]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 5})
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-03-14 14:51:23,822 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,823 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2025-03-14 14:51:23,824 INFO sqlalchemy.engine.Engine [generated in 0.00068s] (5,)
x: 6, y: 8
x: 9, y: 10
2025-03-14 14:51:23,825 INFO sqlalchemy.engine.Engine ROLLBACK


## [データベースメタデータの操作｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/3251b3)

In [9]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [10]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String)
)

In [11]:
print(user_table.c.name)
print(user_table.c.keys())

user_account.name
['id', 'name', 'fullname']


In [12]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

In [13]:
metadata_obj.create_all(engine)

2025-03-14 14:51:23,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,882 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-03-14 14:51:23,883 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,884 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-03-14 14:51:23,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,886 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-03-14 14:51:23,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,887 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-03-14 14:51:23,888 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,889 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-03-14 14:51:23,890 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2025-03-14 14:51:23,891 INFO sqlalchemy.engine.Engine 
C

In [14]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [15]:
from sqlalchemy.orm import Mapped, mapped_column, relationship
from typing import List, Optional


# Userクラスの定義
class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]

    addresses: Mapped[List["Address"]] = relationship(back_populates="user")


# Addressクラスの定義
class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship(back_populates="addresses")

In [16]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2025-03-14 14:51:23,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,936 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2025-03-14 14:51:23,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,938 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-03-14 14:51:23,939 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2025-03-14 14:51:23,941 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2025-03-14 14:51:23,942 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,942 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2025-03-14 14:51:23,943 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-14 14:51:23,944 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [17]:
print(some_table.c.keys())

['x', 'y']


## [データ操作の概要｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/002e99)

In [18]:
from sqlalchemy import insert

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

print(stmt)

with engine.begin() as conn:
    conn.execute(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)
2025-03-14 14:51:23,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,982 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-03-14 14:51:23,983 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ('spongebob', 'Spongebob Squarepants')
2025-03-14 14:51:23,984 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
from sqlalchemy import select

stmt = select(user_table)

with engine.begin() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(row)

2025-03-14 14:51:23,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:23,998 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-03-14 14:51:23,999 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
(1, 'spongebob', 'Spongebob Squarepants')
2025-03-14 14:51:24,001 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
stmt = select(user_table).where(user_table.c.name == "spongebob")

with engine.begin() as conn:
    result = conn.execute(stmt)
    print(result.all())

2025-03-14 14:51:24,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:24,013 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-03-14 14:51:24,014 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ('spongebob',)
[(1, 'spongebob', 'Spongebob Squarepants')]
2025-03-14 14:51:24,016 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
from sqlalchemy import update

stmt = update(user_table).where(user_table.c.name == "spongebob").values(fullname="Spongebob Squarepants")

with engine.begin() as conn:
    conn.execute(stmt)

2025-03-14 14:51:24,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:24,029 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2025-03-14 14:51:24,030 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ('Spongebob Squarepants', 'spongebob')
2025-03-14 14:51:24,031 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
from sqlalchemy import delete

stmt = delete(user_table).where(user_table.c.name == "spongebob")

with engine.begin() as conn:
    conn.execute(stmt)

2025-03-14 14:51:24,041 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:24,043 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.name = ?
2025-03-14 14:51:24,043 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('spongebob',)
2025-03-14 14:51:24,044 INFO sqlalchemy.engine.Engine COMMIT


## [ORMでのデータ操作｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/fa5f6d)

In [23]:
from sqlalchemy import insert

spongebob = User(name="spongebob", fullname="Spongebob Squarepants")
patrik = User(name="patrik", fullname="Patrik Star")

In [24]:
from sqlalchemy.orm import Session

# with Session(engine, expire_on_commit=False) as session:
with Session(engine) as session:
    session.add(spongebob)
    session.add_all([patrik])
    session.commit()

    print(spongebob.id)

2025-03-14 14:51:24,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:24,076 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-03-14 14:51:24,076 INFO sqlalchemy.engine.Engine [generated in 0.00014s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')
2025-03-14 14:51:24,078 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-03-14 14:51:24,079 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('patrik', 'Patrik Star')
2025-03-14 14:51:24,081 INFO sqlalchemy.engine.Engine COMMIT
2025-03-14 14:51:24,082 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-14 14:51:24,085 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-03-14 14:51